In [1]:
from data import*
from keras.layers import *
from keras import backend as K
from keras.models import*
from keras.callbacks import*

In [2]:
dropout_rate = 0.5

def conv_batchnorm_relu_block(input_tensor, nb_filter, kernel_size=3):
    x = Conv2D(nb_filter, (kernel_size, kernel_size), padding='same')(input_tensor)
    x = BatchNormalization(axis=2)(x)
    x = Activation('relu')(x)
    return x

In [3]:
_filter=[32,64,128,256,1024]
global bn_axis
K.set_image_data_format("channels_last")
bn_axis = -1
inputs = Input(shape=(256,256,1))
#Pooling
pool=[]
#卷積
conv_=[]
#輸入層
c=conv_batchnorm_relu_block(inputs, nb_filter=_filter[0])
l=[]
l.append(c)
conv_.append(l)
pool.append(MaxPool2D((2, 2), strides=(2, 2))(c))
#隱藏層
for f in range(1,len(_filter)):
    c=conv_batchnorm_relu_block(pool[f-1], nb_filter=_filter[f])
    l=[]
    l.append(c)
    conv_.append(l)
    pool.append(MaxPool2D((2, 2), strides=(2, 2))(c))
for h in range(0,len(_filter)-1):
    for f in range(1,len(_filter)):
        up=Conv2DTranspose(_filter[f-1], (2, 2), strides=(2, 2), padding='same')(conv_[f][len(conv_[f])-1])
        l=[]
        l.append(up)
        for c in conv_[f-1]:
            l.append(c)
        conv=concatenate(l, axis=bn_axis)
        conv_[f-1].append(conv_batchnorm_relu_block(conv, nb_filter=_filter[f-1]))
conv=[]
for c2 in conv_[0]:
    conv.append(c2)
resultConv=concatenate(conv, axis=bn_axis)
resultConv = conv_batchnorm_relu_block(resultConv, nb_filter=_filter[0])
model = Model(inputs=inputs, outputs=Conv2D(1, (1, 1), activation='sigmoid', padding='same')(resultConv))

In [4]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
b=1
train = trainGenerator(b,r'data\folder3','train','trainmask',data_gen_args,save_to_dir = None)
val= trainGenerator(b,r'data\folder3','validation','validationmask',data_gen_args,save_to_dir = None)

In [5]:
model_checkpoint = ModelCheckpoint('u-net++_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model_checkpoint2 = EarlyStopping(monitor='loss', patience=5,verbose=1)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4), loss = 'binary_crossentropy', metrics = [tf.keras.metrics.IoU(num_classes=2, target_class_ids=[0])])
model.fit(train,steps_per_epoch=5000,epochs=30,callbacks=[model_checkpoint,model_checkpoint2],validation_steps=val)

Found 3627 images belonging to 1 classes.
Found 3627 images belonging to 1 classes.
Epoch 1/30
5000/5000 [==============================] - ETA: 0s - loss: 0.0432 - io_u: 0.9845
Epoch 1: loss improved from inf to 0.04322, saving model to u-net++_membrane.hdf5
5000/5000 [==============================] - 399s 78ms/step - loss: 0.0432 - io_u: 0.9845
Epoch 2/30
5000/5000 [==============================] - ETA: 0s - loss: 0.0252 - io_u: 0.9845
Epoch 2: loss improved from 0.04322 to 0.02518, saving model to u-net++_membrane.hdf5
5000/5000 [==============================] - 392s 78ms/step - loss: 0.0252 - io_u: 0.9845
Epoch 3/30
5000/5000 [==============================] - ETA: 0s - loss: 0.0214 - io_u: 0.9845
Epoch 3: loss improved from 0.02518 to 0.02136, saving model to u-net++_membrane.hdf5
5000/5000 [==============================] - 394s 79ms/step - loss: 0.0214 - io_u: 0.9845
Epoch 4/30
5000/5000 [==============================] - ETA: 0s - loss: 0.0178 - io_u: 0.9844
Epoch 4: loss im

In [15]:
model.fit(train,steps_per_epoch=5000,epochs=1,callbacks=[model_checkpoint,model_checkpoint2],validation_steps=val)

5000/5000 [==============================] - ETA: 0s - loss: 0.0054 - io_u: 0.9845
Epoch 1: loss improved from 0.00554 to 0.00541, saving model to u-net++_membrane.hdf5
5000/5000 [==============================] - 395s 79ms/step - loss: 0.0054 - io_u: 0.9845


In [17]:
pPath=r"data\folder4\predict"
imgCount=46
test = testGenerator(r"data\folder4\test",imgCount)
results = model.predict_generator(test,imgCount,verbose=1)
saveResult(pPath,results)

 1/46 [..............................] - ETA: 1s

C:\Users\user\AppData\Local\Temp\ipykernel_19816\1722158046.py:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  results = model.predict_generator(test,imgCount,verbose=1)


46/46 [==============================] - 1s 22ms/step
